In [1]:
pip install langchain==0.2.15 langchain-community==0.2.15 langchain-core==0.2.43 langchain-google-genai==1.0.9 langchain-text-splitters>=0.2.0 faiss-cpu>=1.8.0 wikipedia

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.2.43 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
google-adk 1.21.0 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-pytho

In [2]:
import os
import wikipedia
from typing import List

In [3]:
# --- Modern LangChain Imports ---
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [4]:
# --- CONFIGURATION ---
# Ensure your API key is set
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY_HERE"

In [5]:
# Initialize Models
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
print("🏗️ PHASE 1: Building Knowledge Bases...")

# 1. Create Dummy "Harry Potter" Content (The "Book")
book_text = """
Harry Potter is a wizard who attends Hogwarts School of Witchcraft and Wizardry.
He was famous from the age of one for surviving the killing curse cast by Lord Voldemort.
In his first year, Harry discovers that the Sorcerer's Stone (also known as the Philosopher's Stone) produces the Elixir of Life.
The Stone was created by the alchemist Nicolas Flamel, who was a partner of Albus Dumbledore.
Nicolas Flamel was a real historical figure born in France, though his magical immortality is a legend.
Harry's best friends are Hermione Granger and Ron Weasley.
They belong to Gryffindor House, which values bravery and daring.
"""

🏗️ PHASE 1: Building Knowledge Bases...


In [7]:
# FIX 1: Split text properly and pass a LIST to FAISS
texts = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_text(book_text)
hp_vector_store = FAISS.from_texts(texts, embeddings_model)

In [8]:
# 2. Archivist Long-Term Memory (Starts Empty)
memory_vector_store = FAISS.from_texts(["The Archivist's memory is currently empty."], embeddings_model)

In [9]:
# --- PHASE 2: TOOL DEFINITIONS ---

@tool
def book_search_tool(query: str) -> str:
    """Searches the local 'Harry Potter' book knowledge base for facts."""
    retriever = hp_vector_store.as_retriever(search_kwargs={"k": 1})
    docs = retriever.invoke(query)
    if docs:
        return f"Found in book: {docs[0].page_content}"
    return "Fact not found in the book."

@tool
def wiki_tool(entity: str) -> str:
    """Searches Wikipedia to verify if a person, place, or concept is real."""
    try:
        # FIX 2: Use the REAL Wikipedia API instead of mocking it
        return wikipedia.summary(entity, sentences=1)
    except wikipedia.exceptions.PageError:
        return f"Wikipedia Denies: {entity} does not exist in our records."
    except wikipedia.exceptions.DisambiguationError:
        return f"Wikipedia Ambiguous: Multiple results found for {entity}."
    except Exception as e:
        return f"Wikipedia Error: {str(e)}"

@tool
def memory_tool(fact: str) -> str:
    """Saves a fully verified, true-in-the-real-world fact into the Archivist's permanent memory."""
    # FIX 3: add_texts only takes the text list, not the embedding model (it already has it)
    memory_vector_store.add_texts([fact])
    return f"SUCCESS: Verified fact saved to long-term memory: '{fact}'"

tools = [book_search_tool, wiki_tool, memory_tool]

In [10]:
# --- PHASE 3: AGENT ORCHESTRATION ---

# FIX 4: Use a standard ReAct PromptTemplate
prompt_template = PromptTemplate.from_template(
    """
    You are the Skeptical Archivist. Your directive is: "Trust, but Verify."

    TOOLS:
    ------
    You have access to the following tools:

    {tools}

    PROCESS:
    --------
    1. Always start by using the 'book_search_tool' to find the claim in the text.
    2. If found, identify the key entity (person, place, object).
    3. Use 'wiki_tool' to verify if that entity exists in the REAL world.
    4. ONLY if Wikipedia confirms it is real, use 'memory_tool' to save it.
    5. If Wikipedia denies it (it's fictional), inform the user you cannot archive it.

    To use a tool, please use the following format:

    ```
    Thought: Do I need to use a tool? Yes
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ```

    When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

    ```
    Thought: Do I need to use a tool? No
    Final Answer: [your response here]
    ```

    Begin!

    New Input: {input}
    {agent_scratchpad}
    """
)

In [11]:
# FIX 5: Use create_react_agent
agent = create_react_agent(llm, tools, prompt_template)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [12]:
# --- EXECUTION ---
if __name__ == "__main__":
    print("\n--- SCENARIO 1: Nicolas Flamel (Real) ---")
    agent_executor.invoke({"input": "Who is Nicolas Flamel?"})

    print("\n--- SCENARIO 2: Lumos Spell (Fictional) ---")
    agent_executor.invoke({"input": "What does the spell Lumos do?"})


--- SCENARIO 1: Nicolas Flamel (Real) ---


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: book_search_tool
Action Input: "Nicolas Flamel"Found in book: Harry Potter is a wizard who attends Hogwarts School of Witchcraft and Wizardry.
He was famous from the age of one for surviving the killing curse cast by Lord Voldemort.
In his first year, Harry discovers that the Sorcerer's Stone (also known as the Philosopher's Stone) produces the Elixir of Life.
The Stone was created by the alchemist Nicolas Flamel, who was a partner of Albus Dumbledore.
Nicolas Flamel was a real historical figure born in France, though his magical immortality is a legend.
Harry's best friends are Hermione Granger and Ron Weasley.
They belong to Gryffindor House, which values bravery and daring.Do I need to use a tool? Yes
Action: wiki_tool
Action Input: Nicolas FlamelWikipedia Denies: Nicolas Flamel does not exist in our records.Do I need to use a tool? No
Final Answer: I 

In [13]:

# --- EXAMPLE WORKFLOW (Conceptual) ---

print("--- TESTING SCENARIO 1: VERIFIABLE FACT (Nicolas Flamel) ---")
result_flamel = agent_executor.invoke({"input": "Who is Nicolas Flamel?"})
# Expected output: BookSearch -> WikiTool -> MemoryTool -> Final Answer
print(f"\nFinal Agent Response: {result_flamel['output']}\n")

print("--- TESTING SCENARIO 2: FICTIONAL FACT (The spell Lumos) ---")
result_lumos = agent_executor.invoke({"input": "What does the spell Lumos do?"})
# Expected output: BookSearch -> WikiTool (fails) -> Final Answer (no MemoryTool call)
print(f"\nFinal Agent Response: {result_lumos['output']}\n")

--- TESTING SCENARIO 1: VERIFIABLE FACT (Nicolas Flamel) ---


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: book_search_tool
Action Input: "Nicolas Flamel"Found in book: Harry Potter is a wizard who attends Hogwarts School of Witchcraft and Wizardry.
He was famous from the age of one for surviving the killing curse cast by Lord Voldemort.
In his first year, Harry discovers that the Sorcerer's Stone (also known as the Philosopher's Stone) produces the Elixir of Life.
The Stone was created by the alchemist Nicolas Flamel, who was a partner of Albus Dumbledore.
Nicolas Flamel was a real historical figure born in France, though his magical immortality is a legend.
Harry's best friends are Hermione Granger and Ron Weasley.
They belong to Gryffindor House, which values bravery and daring.Do I need to use a tool? Yes
Action: wiki_tool
Action Input: Nicolas FlamelWikipedia Denies: Nicolas Flamel does not exist in our records.Do I need to use a tool? N